This notebook computes all of the "distances" (the fuzzy logic scores for IOU, DICE, ...) between the GradCam maps produced by the final models and the visual characterstics maps created by the dermatologists. That is: 

```
for each image i  
  for each GradCam image i_gc  
    for each dermatologist d  
      for each characteristic d_char  
        compute the visual distance between i_gc and d_char 
```


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt 
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [98]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
    'derm8',
]

characteristics = [
    'Closed comedo',
    'Cyst',
    'Dermatoglyph disruption',
    'Leukotrichia',
    'Macule',
    'Nodule',
    'Open comedo',
    'Papule',
    'Patch',
    'Plaque',
    'Pustule',
    'Scale',
    'Scar',
    'Sun damage',
    'Telangiectasia',
    'Thrombosed capillaries'
]

images_path = Path('/home/ubuntu/hot-store/dermx/all_images')
results_path = Path('/home/ubuntu/hot-store/final/')

# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics


def calculate_mask_metrics(gradcam_path, mask_path, interpolation_method=Image.BICUBIC):
    """
    Calculates the fuzzy logic metrics given the paths to a pair of input images.
    The derm mask is resized to match the size of the gradcam image.
    
    Input:
    - gradcam_image_path: Pathlib path to a gradCam image. The file is assumed to be in .npy format.
    - derm_char_mask_path: Pathlib path to a derm annotation. The file is assumed to be in a format that can
                           be opened by PIL.
    - interpolation_method: String. The method used for interpolation when resizing the derm mask. Options are
                            NEAREST, BOX, BILINEAR, HAMMING, BICUBIC, LANCZOS. Default is NEAREST.    
    """
    
    # Open images.
    gradcam = Image.fromarray(np.load(gradcam_path, allow_pickle=True))
    mask = np.asarray(Image.open(mask_path)) / 255
    
    # Resize the derm mask if its size does not match the size of the gradcam image.
    if gradcam.size[::-1] != mask.shape:
        # Note that resize uses (cols, rows) format, while .shape is in (rows, cols) format
        gradcam = gradcam.resize((mask.shape[1], mask.shape[0]), interpolation_method)

    # Normalize to 0-1
    gradcam = np.asarray(gradcam)
    if gradcam.min() != gradcam.max():
        gradcam = (gradcam - gradcam.min()) / (gradcam.max() - gradcam.min())
    
    return pixel_metrics_fuzzy(mask, gradcam)


## Per image metrics

In [99]:
def get_per_image_metrics(results_path, model_name, images_path, masks_path, classes, columns):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.BICUBIC

    # Compute metrics for each image
    for image_path in image_paths:
        for diagnosis in classes:
            # Check if matching GradCam file exists.
            gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
            if gradcam_path.is_file():
                mask_path = masks_path / Path(image_path.stem + '.png') 
                if mask_path.is_file():
                    # Calculate the value of the metics given the GradCam image and the derm mask.
                    gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                    gradcam_metric_val['gradcam_class'] = diagnosis 
                    gradcam_metric_val['filename'] = image_path.stem 
                    metrics_list.append(gradcam_metric_val)
            else:
                print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [100]:
def analyse_per_image_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
):
    per_image_metrics_df = get_per_image_metrics(results_path, model_name, images_path, masks_path, classes, columns)
    
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]
    
    return {
        'iou': preds_df['iou'].mean(),
        'dice': preds_df['dice'].mean(),
        'precision': preds_df['precision'].mean(),
        'recall': preds_df['recall'].mean(),
        'negative_predictive_value': preds_df['negative_predictive_value'].mean(),
        'specificity': preds_df['specificity'].mean(), 
    }


In [101]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_image')
columns = [
         'filename',
         'gradcam_class',
         'iou',
         'dice',
         'precision',
         'recall',
         'negative_predictive_value',
         'specificity'
    ]

models_performance = []
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}'
        preds_name = f'dermx_{idx}_preds.csv'
        print(model_name)
        model_performance = analyse_per_image_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes)
        model_performance['model_architecture'] = model['model_architecture']
        model_performance['model_name'] = model['model_name']
        models_performance.append(model_performance)
per_image_models_performance_df = pd.DataFrame.from_records(models_performance, columns=models_performance[0].keys())   

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


In [102]:
per_image_models_performance_df.groupby('model_architecture').describe().to_csv('per_image_models_performance.csv')

In [103]:
per_image_models_performance_df.groupby('model_architecture').describe()

iou                                                    \
                   count      mean       std       min       25%       50%   
model_architecture                                                           
efficientnet         5.0  0.271429  0.007869  0.262124  0.266887  0.270617   
inception            5.0  0.287202  0.003044  0.283190  0.285676  0.286859   
inceptionresnet      5.0  0.233898  0.004144  0.228962  0.232029  0.232174   
mobilenetv1          5.0  0.248671  0.004247  0.244374  0.245689  0.247183   
mobilenetv2          5.0  0.265612  0.012531  0.255311  0.257100  0.263311   
nasnetmobile         5.0  0.298846  0.004624  0.293082  0.297910  0.298216   
resnet               5.0  0.193737  0.016560  0.180114  0.180219  0.184849   
resnetv2             5.0  0.265052  0.005316  0.258138  0.262075  0.264816   
vgg                  5.0  0.204173  0.037470  0.154391  0.180891  0.205357   
xception             5.0  0.298458  0.005703  0.292067  0.292963  0.300516   

                                        dice                                \
                         75%       max count      mean       std       min   
model_architecture                                                           
efficientnet        0.274737  0.282781   5.0  0.406160  0.010054  0.393621   
inception           0.289353  0.290931   5.0  0.429913  0.002853  0.426432   
inceptionresnet     0.237300  0.239026   5.0  0.362842  0.005428  0.357054   
mobilenetv1         0.251607  0.254499   5.0  0.382829  0.005667  0.376993   
mobilenetv2         0.265651  0.286689   5.0  0.402867  0.015747  0.390393   
nasnetmobile        0.299037  0.305986   5.0  0.440782  0.006438  0.432864   
resnet              0.211212  0.212291   5.0  0.303811  0.022599  0.283083   
resnetv2            0.268527  0.271705   5.0  0.406156  0.006480  0.398147   
vgg                 0.237770  0.242456   5.0  0.324631  0.050569  0.256352   
xception            0.301568  0.305176   5.0  0.443321  0.006912  0.435826   

                                                           precision  \
                         25%       50%       75%       max     count   
model_architecture                                                     
efficientnet        0.400945  0.406296  0.409281  0.420655       5.0   
inception           0.428346  0.429872  0.430871  0.434043       5.0   
inceptionresnet     0.359412  0.360559  0.367802  0.369382       5.0   
mobilenetv1         0.378148  0.381799  0.387061  0.390145       5.0   
mobilenetv2         0.392161  0.399949  0.402267  0.429564       5.0   
nasnetmobile        0.439102  0.439615  0.441639  0.450691       5.0   
resnet              0.287684  0.291858  0.325121  0.331310       5.0   
resnetv2            0.401748  0.406018  0.410769  0.414098       5.0   
vgg                 0.293605  0.328296  0.369505  0.375397       5.0   
xception            0.436313  0.446004  0.447141  0.451323       5.0   

                                                                      \
                        mean       std       min       25%       50%   
model_architecture                                                     
efficientnet        0.392285  0.009650  0.380509  0.386192  0.391279   
inception           0.428233  0.008992  0.413295  0.426202  0.433659   
inceptionresnet     0.417700  0.008222  0.406083  0.413344  0.419124   
mobilenetv1         0.381538  0.007146  0.374367  0.376523  0.378422   
mobilenetv2         0.442417  0.019842  0.429076  0.432302  0.432894   
nasnetmobile        0.416953  0.009891  0.405167  0.407894  0.419831   
resnet              0.349273  0.009836  0.338299  0.343084  0.346747   
resnetv2            0.503757  0.007440  0.493189  0.498822  0.507513   
vgg                 0.463737  0.020234  0.449040  0.449163  0.458262   
xception            0.487092  0.007303  0.476710  0.483172  0.489484   

                                       recall                                \
                         75%       max

## Per characteristic metrics

In [104]:
def get_per_characteristic_metrics(results_path, model_name, images_path, masks_path, classes, columns, characteristics):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.NEAREST

    # Compute metrics for each image
    for image_path in image_paths:
        for characteristic in characteristics:
            for diagnosis in classes:
                # Check if matching GradCam file exists.
                gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
                if gradcam_path.is_file():
                    mask_path = masks_path / Path(f'{characteristic}_{image_path.stem}.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                        gradcam_metric_val['gradcam_class'] = diagnosis 
                        gradcam_metric_val['characteristic'] = characteristic 
                        gradcam_metric_val['filename'] = image_path.stem 
                        metrics_list.append(gradcam_metric_val)
                else:
                    print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [105]:
def analyse_per_characteristics_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
    characteristics,
    metrics
):
    per_image_metrics_df = get_per_characteristic_metrics(results_path, model_name, images_path, masks_path, classes, columns, characteristics)
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]

    metrics_dict = {}
    for characteristic in characteristics:
        metrics_dict[characteristic] = {}
        for metric in metrics:
            metrics_dict[characteristic][metric] = preds_df[preds_df['characteristic'] == characteristic][metric].mean()
            
    return metrics_dict


In [106]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_characteristic')
columns = [
    'filename',
    'gradcam_class',
    'characteristic',
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]
metrics = [
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]

models_performance_dict = {}
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}'
        preds_name = f'dermx_{idx}_preds.csv'
        print(model_name)
        model_performance = analyse_per_characteristics_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes, characteristics, metrics)
        for k, v in model_performance.items():
            models_performance_dict[(model['model_architecture'], model_name, k)] = v

per_characteristic_models_performance_df = pd.DataFrame(models_performance_dict).T

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


In [107]:
per_characteristic_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe().to_csv('per_characteristic_models_performance.csv')

In [108]:
per_characteristic_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe()

iou                                \
                                        count      mean       std       min   
level_0         level_2                                                       
efficientnet    Closed comedo             5.0  0.131270  0.007432  0.124035   
                Cyst                      5.0  0.020761  0.002873  0.017513   
                Dermatoglyph disruption   0.0       NaN       NaN       NaN   
                Leukotrichia              5.0  0.084604  0.012083  0.069980   
                Macule                    5.0  0.123917  0.009282  0.110930   
                Nodule                    5.0  0.036698  0.003554  0.032145   
                Open comedo               5.0  0.123532  0.003325  0.118561   
                Papule                    5.0  0.100653  0.006561  0.092829   
                Patch                     5.0  0.141276  0.005045  0.134399   
                Plaque                    5.0  0.202840  0.004233  0.198168   
                Pustule                   5.0  0.022590  0.001744  0.020431   
                Scale                     5.0  0.189437  0.003664  0.184018   
                Scar                      5.0  0.203301  0.007962  0.194810   
                Sun damage                5.0  0.260801  0.017480  0.233330   
                Telangiectasia            5.0  0.110675  0.002797  0.107297   
                Thrombosed capillaries    0.0       NaN       NaN       NaN   
inception       Closed comedo             5.0  0.130675  0.006548  0.123543   
                Cyst                      5.0  0.017516  0.000929  0.016237   
                Dermatoglyph disruption   0.0       NaN       NaN       NaN   
                Leukotrichia              2.0  0.062460  0.038566  0.035190   
                Macule                    5.0  0.132655  0.004078  0.126128   
                Nodule                    5.0  0.040725  0.004204  0.034136   
                Open comedo               5.0  0.125051  0.010046  0.114238   
                Papule                    5.0  0.103667  0.001975  0.101690   
                Patch                     5.0  0.175425  0.009285  0.162766   
                Plaque                    5.0  0.226699  0.005309  0.219297   
                Pustule                   5.0  0.021814  0.000774  0.020922   
                Scale                     5.0  0.215003  0.003395  0.209698   
                Scar                      5.0  0.216704  0.008673  0.208751   
                Sun damage                5.0  0.245159  0.005450  0.238721   
                Telangiectasia            5.0  0.126947  0.015776  0.112620   
                Thrombosed capillaries    2.0  0.048638  0.000583  0.048226   
inceptionresnet Closed comedo             5.0  0.101636  0.012946  0.085783   
                Cyst                      5.0  0.016872  0.002754  0.014852   
                Dermatoglyph disruption   0.0       NaN       NaN       NaN   
                Leukotrichia              5.0  0.069659  0.045322  0.029405   
                Macule                    5.0  0.102155  0.010699  0.090664   
                Nodule                    5.0  0.030430  0.001923  0.028216   
                Open comedo               5.0  0.113674  0.005953  0.103893   
                Papule                    5.0  0.083186  0.009170  0.073266   
                Patch                     5.0  0.127150  0.006411  0.118040   
                Plaque                    5.0  0.175048  0.005764  0.165849   
                Pustule                   5.0  0.020760  0.004032  0.018084   
                Scale                     5.0  0.175499  0.009061  0.160178   
                Scar                      5.0  0.160927  0.017876  0.144392   
                Sun damage                5.0  0.193098  0.010840  0.174251   
                Telangiectasia            5.0  0.089162  0.014228  0.071959   
                Thrombosed capillaries    1.0  0.002481       NaN  0.002481   
mobilenetv1  

## Per derm

In [109]:
def get_per_derm_metrics(results_path, model_name, images_path, masks_path, classes, columns, derms):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.NEAREST

    # Compute metrics for each image
    for image_path in image_paths:
        for derm in derms:
            for diagnosis in classes:
                # Check if matching GradCam file exists.
                gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
                if gradcam_path.is_file():
                    mask_path = masks_path / Path(f'{derm}_{image_path.stem}.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                        gradcam_metric_val['gradcam_class'] = diagnosis 
                        gradcam_metric_val['derm'] = derm
                        gradcam_metric_val['filename'] = image_path.stem 
                        metrics_list.append(gradcam_metric_val)
                else:
                    print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [110]:
def analyse_per_derms_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
    derms,
    metrics
):
    per_image_metrics_df = get_per_derm_metrics(results_path, model_name, images_path, masks_path, classes, columns, derms)
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]
    metrics_dict = {}
    for derm in derms:
        metrics_dict[derm] = {}
        for metric in metrics:
            metrics_dict[derm][metric] = preds_df[preds_df['derm'] == derm][metric].mean()
            
    return metrics_dict


In [111]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_derm')
columns = [
    'filename',
    'gradcam_class',
    'derm',
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]
metrics = [
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]

models_performance_dict = {}
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}'
        preds_name = f'dermx_{idx}_preds.csv'
        print(model_name)
        model_performance = analyse_per_derms_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes, derms, metrics)

        for k, v in model_performance.items():
            models_performance_dict[(model['model_architecture'], model_name, k)] = v

per_derm_models_performance_df = pd.DataFrame(models_performance_dict).T

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in double_scalars


In [112]:
per_derm_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe().to_csv('per_derm_models_performance.csv')

In [ ]:
per_derm_models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe()


### Old

In [2]:
# Set paths

# Path to the GradCam images.
gradcam_main_path = Path('/home/ubuntu/store/efficientnet-final-size/visualisation/gradcam/')

# Set path to the model used to create the visualisations.
model_path = Path('efficientnetb4_0')

# Path to the masks create by the derms for the individual characteristics.
derm_mask_main_path = Path('/home/ubuntu/store/masks/masks_resized')

# Path to the rescaled test images.
test_im_path = Path('/home/ubuntu/store/DermX-test-set/test/')


In [3]:
full_gradcam_path = gradcam_main_path / model_path
full_lime_path = lime_main_path / model_path

In [4]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm0',
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
]

chars = [
    'closed-comedo',
    'cyst',
    'dermatoglyph-disruption',
    'leukotrichia',
    'macule',
    'nodule',
    'open-comedo',
    'papule',
    'patch',
    'plaque',
    'pustule',
    'scale',
    'scar',
    'sun-damage',
    'telangiectasia',
    'thrombosed-capillaries'
]


In [5]:
# Extract the paths to the derm masks. The resulting list is only used for a sanity check.
derm_mask_paths = [p for p in derm_mask_main_path.iterdir() if p.suffix == '.png']
derm_mask_paths[0:5]

[PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043023HB_mihaela_2021-05-27-masks_plaque.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043212HB_adelina_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043269HB_adelina_2021-05-27-masks_papule.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/293--WatermarkedWyJXYXRlcm1hcmtlZCJd_mihaela_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/viral-wart-08--WatermarkedWyJXYXRlcm1hcmtlZCJd_oana_2021-05-27-masks_papule.png')]

In [6]:
# Extract the paths to the test images. Only needed for visualization and debugging.
test_img_paths = [p for p in Path(test_im_path).rglob('*.jpeg')]
test_img_paths[0:5]

[PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/017359HB.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/476--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/469--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3742--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3753--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg')]

In [7]:
# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics

In [8]:
def calc_res(gradcam_image_path, derm_char_mask_path, interpolation_method=Image.NEAREST):
    """
    Calculates the fuzzy logic metrics given the paths to a pair of input images.
    The derm mask is resized to match the size of the gradcam image.
    
    Input:
    - gradcam_image_path: Pathlib path to a gradCam image. The file is assumed to be in .npy format.
    - derm_char_mask_path: Pathlib path to a derm annotation. The file is assumed to be in a format that can
                           be opened by PIL.
    - interpolation_method: String. The method used for interpolation when resizing the derm mask. Options are
                            NEAREST, BOX, BILINEAR, HAMMING, BICUBIC, LANCZOS. Default is NEAREST.    
    """
    
    # Open images.
    gradcam_im = np.load(gradcam_image_path, allow_pickle=True)
    mask_im = Image.open(derm_char_mask_path)
    
    # Resize the derm mask if its size does not match the size of the gradcam image.
    if gradcam_im.shape != mask_im.size[::-1]:
        # Note that resize uses (cols, rows) format, while .shape is in (rows, cols) format.
        mask_im = mask_im.resize((gradcam_im.shape[::-1]), interpolation_method)
    
    # Corvert the derm mask to numpy format and normalize to [0, 1].
    mask_im = np.asarray(mask_im) / 255
  
    res = pixel_metrics_fuzzy(mask_im, gradcam_im)
    return res

In [9]:
# For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
# class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
# As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
rec_dd = lambda: defaultdict(rec_dd)
out_gradcam = rec_dd()
hit_counter = 0

# Set the method used for interpolation when resizing the derm annotations.
interpolation = Image.NEAREST

for p in test_img_paths:
    for c in classes:
        # Check if matching GradCam file exists.
        gc_path = full_gradcam_path / Path(p.stem + '_' + c + '.npy')
        if gc_path.is_file():
            for d in derms:
                for ch in chars:
                    # Check if this derm has created a mask for this characteristic.
                    mask_path = derm_mask_main_path / Path(p.stem + '_' + d + '_2021-05-27-masks_' + ch + '.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calc_res(gc_path, mask_path, interpolation)
                        out_gradcam[p.stem][c][d][ch] = gradcam_metric_val
                        hit_counter += 1
        else:
            print('GradCam or LIME file missing for image: ', p)

print(hit_counter)



19578


In [12]:
# We expect that the value of the hit counter should be equal to the number of derm masks multiplied with the number
# of classes.
if len(derm_mask_paths)*6 != hit_counter:
    print('Oh no, some files were not found. Expected/found: ', len(derm_mask_paths)*6, hit_counter)
else:
    print('GradCam images found for all derm annotations.')


GradCam images found for all derm annotations.


In [13]:
def transform_to_list(nested_dict):
    # Transform the gradcam defaultdict to a list of tuples.
    
    out = []
    for im_name, class_dict in nested_dict.items():
        for class_name, derm_dict in class_dict.items():
            for derm_name, char_dict in derm_dict.items():
                for char_name, metric
                _dict in char_dict.items():
                    tmp = tuple(metric_dict.values())
                    out.append( (im_name, class_name, derm_name, char_name) + tmp )
    return out

In [14]:
gradcam_res_list = transform_to_list(out_gradcam)
len(gradcam_res_list)

19578

## Make output dataFrames/csv files

In [18]:
col_names = ['image_name',
             'visualisation_class',
             'derm',
             'characteristic',
             'iou',
             'dice',
             'precision',
             'recall',
             'negative_predictive_value',
             'specificity'
            ]

In [ ]:
gradcam_df = pd.DataFrame.from_records(gradcam_res_list, columns=col_names)
gradcam_df

In [22]:
# Save dataFrames
model_name = str(model_path)
gradcam_df.to_csv(model_name + "_gradcam_scores.csv")


# Filter

In [123]:
import glob
model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')

In [124]:
model_names

['/home/ubuntu/store/efficientnet-final-size/efficientnetb4_4.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_0.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_2.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_3.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_1.h5']

In [125]:
model_name = Path(model_names[1]).stem

In [126]:
def get_table(results_pred):
    results_mean = results_pred.groupby('characteristic').median()
    results_std =  results_pred.groupby('characteristic').mad()
    
    ### Add means
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean']   = results_mean.mean()
    
    columns = results_mean.columns.to_list()

    table_pred = results_mean
    return table_pred

class_map = {'0' : 'Acne' ,
             '1' : 'Actinic keratosis',
             '2' : 'Psoriasis' ,
             '3' : 'Seborrheic dermatitis',
             '4' : 'Viral warts',
             '5' : 'Vitiligo'}

filefolder = model_name + "_gradcam_scores.csv"
predsfile = '/home/ubuntu/store/efficientnet-final-size/' + model_name + '_preds.csv'

In [127]:
filtered_subjects = pd.read_csv('./include_images_525.csv')
filtered_subjects['image_id'] = filtered_subjects['image_id'].apply(lambda x: Path(x).stem)
filtered_subjects = filtered_subjects.rename(columns={'image_id': 'image_name'})
filtered_subjects = filtered_subjects.drop(columns=['Unnamed: 0'])

In [128]:
df_preds = pd.read_pickle(predsfile)
gradcam_df = pd.read_csv(filefolder)
gradcam_df=gradcam_df.drop(['Unnamed: 0'], axis=1)

In [129]:
df_preds['pred'] =  df_preds['pred'].apply(lambda x: class_map[str(x)])
df_preds['actual'] = df_preds['actual'].apply(lambda x: class_map[str(x)])
df_preds['filenames'] = df_preds['filenames'].apply(lambda x: Path(x).stem)
df_preds = df_preds.rename(columns={'filenames':'image_name'})
df_preds = df_preds.merge(filtered_subjects, on = 'image_name')

In [32]:
result = pd.merge(gradcam_df,df_preds, on = 'image_name')

results_pred = result[result.visualisation_class == result.pred]
table_pred = get_table(results_pred)
# Get it for actual
results_actual = result[result.visualisation_class == result.actual  ]
table_actual = get_table(results_actual)

table_pred.to_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl')
table_actual.to_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl')

results_equal = result[ (result.actual == result.pred) & (result.visualisation_class == result.pred) ]
table_equal = get_table(results_equal)

results_diff = result[ (result.actual != result.pred) & (result.visualisation_class == result.pred) ]
table_diff = get_table(results_diff)

table_equal.to_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl')
table_diff.to_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl')

# Creating Benchmarks

## By concatenation of all models

In [89]:
def lt_format(results_performance, col_n):
    results_mean = results_performance.groupby(col_n).mean()
    results_std = results_performance.groupby(col_n).std()
    
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean'] = results_std.mean()
    for col in results_performance.columns.to_list():
        results_mean[col] = results_mean[col].apply(lambda x: f'{np.round(x,decimals=2)} $\pm$ ')
        results_std[col] = results_std[col].apply(lambda x: f'{np.round(x,decimals=2)}')
    
    return results_mean + results_std

model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')
results_actual = pd.DataFrame()
results_pred = pd.DataFrame()
results_equal = pd.DataFrame()
results_diff = pd.DataFrame()
results_performance = pd.DataFrame()

for mn in sorted(model_names):
    model_name = Path(mn).stem
    performance = pd.read_pickle('./' + model_name + '_performance.pkl')
    actual = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl').loc[:,['recall','specificity','dice']]
    pred = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl').loc[:,['recall','specificity','dice']]
    equal = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl').loc[:,['recall','specificity','dice']]
    diff = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl').loc[:,['recall','specificity','dice']]
    
    performance=performance.drop('mean')
    actual = actual.drop('mean')
    pred = pred.drop('mean')
    equal = equal.drop('mean')
    diff = diff.drop('mean')
    
    results_performance = pd.concat([results_performance, performance])
    results_actual = pd.concat([results_actual, actual])
    results_pred = pd.concat([results_pred, pred]) 
    results_diff = pd.concat([results_diff, diff])

results_performance.index.name = 'label'

table_performance = lt_format(results_performance, 'label')
table_actual = lt_format(results_actual, 'characteristic')

In [90]:
table_performance = table_performance.drop(columns = ['accuracy'])

In [91]:
table_performance

,precision,f1-score,specificity
label,,,
acne,0.77 $\pm$ 0.16,0.65 $\pm$ 0.32,0.97 $\pm$ 0.02
actinic_keratosis,0.74 $\pm$ 0.08,0.55 $\pm$ 0.11,0.96 $\pm$ 0.02
psoriasis_no_pustular,0.43 $\pm$ 0.14,0.57 $\pm$ 0.12,0.67 $\pm$ 0.23
seborrheic_dermatitis,0.62 $\pm$ 0.08,0.45 $\pm$ 0.22,0.95 $\pm$ 0.03
vitiligo,0.0 $\pm$ 0.01,0.0 $\pm$ 0.01,0.9 $\pm$ 0.03
wart,0.08 $\pm$ 0.04,0.07 $\pm$ 0.04,0.86 $\pm$ 0.04
mean,0.44 $\pm$ 0.08,0.38 $\pm$ 0.14,0.89 $\pm$ 0.06


In [87]:
print(table_performance.to_latex())

\begin{tabular}{llll}
\toprule
{} &        precision &         f1-score &      specificity \\
label                 &                  &                  &                  \\
\midrule
acne                  &  0.77 \$\textbackslash pm\$ 0.16 &  0.65 \$\textbackslash pm\$ 0.32 &  0.97 \$\textbackslash pm\$ 0.02 \\
actinic\_keratosis     &  0.74 \$\textbackslash pm\$ 0.08 &  0.55 \$\textbackslash pm\$ 0.11 &  0.96 \$\textbackslash pm\$ 0.02 \\
psoriasis\_no\_pustular &  0.43 \$\textbackslash pm\$ 0.14 &  0.57 \$\textbackslash pm\$ 0.12 &  0.67 \$\textbackslash pm\$ 0.23 \\
seborrheic\_dermatitis &  0.62 \$\textbackslash pm\$ 0.08 &  0.45 \$\textbackslash pm\$ 0.22 &  0.95 \$\textbackslash pm\$ 0.03 \\
vitiligo              &   0.0 \$\textbackslash pm\$ 0.01 &   0.0 \$\textbackslash pm\$ 0.01 &   0.9 \$\textbackslash pm\$ 0.03 \\
wart                  &  0.08 \$\textbackslash pm\$ 0.04 &  0.07 \$\textbackslash pm\$ 0.04 &  0.86 \$\textbackslash pm\$ 0.04 \\
mean                  &  0.44 \

In [39]:
table_actual

,recall,specificity,dice
characteristic,,,
closed-comedo,0.21 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.03
cyst,0.2 $\pm$ 0.14,0.85 $\pm$ 0.03,0.02 $\pm$ 0.01
dermatoglyph-disruption,0.09 $\pm$ 0.05,0.79 $\pm$ 0.06,0.05 $\pm$ 0.02
leukotrichia,0.14 $\pm$ 0.1,0.82 $\pm$ 0.04,0.07 $\pm$ 0.03
macule,0.27 $\pm$ 0.07,0.8 $\pm$ 0.03,0.09 $\pm$ 0.03
nodule,0.2 $\pm$ 0.08,0.82 $\pm$ 0.03,0.03 $\pm$ 0.01
open-comedo,0.19 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.04
papule,0.23 $\pm$ 0.07,0.8 $\pm$ 0.03,0.07 $\pm$ 0.01
patch,0.28 $\pm$ 0.06,0.8 $\pm$ 0.03,0.21 $\pm$ 0.04


In [40]:
print(table_actual[['dice','recall','specificity']].to_latex())

\begin{tabular}{llll}
\toprule
{} &             dice &           recall &      specificity \\
characteristic          &                  &                  &                  \\
\midrule
closed-comedo           &  0.08 \$\textbackslash pm\$ 0.03 &   0.21 \$\textbackslash pm\$ 0.1 &  0.83 \$\textbackslash pm\$ 0.04 \\
cyst                    &  0.02 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.14 &  0.85 \$\textbackslash pm\$ 0.03 \\
dermatoglyph-disruption &  0.05 \$\textbackslash pm\$ 0.02 &  0.09 \$\textbackslash pm\$ 0.05 &  0.79 \$\textbackslash pm\$ 0.06 \\
leukotrichia            &  0.07 \$\textbackslash pm\$ 0.03 &   0.14 \$\textbackslash pm\$ 0.1 &  0.82 \$\textbackslash pm\$ 0.04 \\
macule                  &  0.09 \$\textbackslash pm\$ 0.03 &  0.27 \$\textbackslash pm\$ 0.07 &   0.8 \$\textbackslash pm\$ 0.03 \\
nodule                  &  0.03 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.08 &  0.82 \$\textbackslash pm\$ 0.03 \\
open-comedo          